# Timing both versions of calculating geodesic distances (neighbours from sklearn vs neighbours from pynndescent)

In [1]:
import numpy as np
import time
import pynndescent
import sklearn.datasets
from sklearn.datasets import fetch_california_housing

### data

The data used for this test is the california housing dataset of sklearn.
I decided to use the first 8000 data points.

In [2]:
X_california, y_california = fetch_california_housing(return_X_y=True)

y_california = y_california[:8000]
X_california = X_california[:8000]
X_california.shape

(8000, 8)

## Timing the first version of geodesic distances (with nearest neighbours from sklearn)

In [3]:
import warnings
import numpy as np
from scipy.sparse import issparse
from scipy.sparse.csgraph import shortest_path
from scipy.sparse.csgraph import connected_components

from sklearn.neighbors import NearestNeighbors, kneighbors_graph
from sklearn.neighbors import radius_neighbors_graph
from sklearn.utils.graph import _fix_connected_components


def compute_geodesic_distances_timing(
    X, 
    n_neighbors=10, 
    radius=None,
    path_method="auto", 
    neighbors_algorithm="auto", 
    n_jobs=None, 
    metric="minkowski",
    p=2, 
    metric_params=None):
    #################################
    start_time_overall = time.time()
    #################################

    #################################
    start_time_find_neigh = time.time()
    #################################

    # first step: find the nearest neighbours
    nbrs_ = NearestNeighbors(
        n_neighbors=n_neighbors,
        radius=radius,
        algorithm=neighbors_algorithm,
        metric=metric,
        p=p,
        metric_params=metric_params,
        n_jobs=n_jobs,
    )
    nbrs_.fit(X)

    # two options: 
    # option 1: choose the number of neighbours
    if n_neighbors is not None:
        nbg = kneighbors_graph(
            nbrs_,
            n_neighbors,
            metric=metric,
            p=p,
            metric_params=metric_params,
            mode="distance",
            n_jobs=n_jobs,
        )
    
    # option 2: choose a radius
    else:
        nbg = radius_neighbors_graph(
            nbrs_,
            radius=radius,
            metric=metric,
            p=p,
            metric_params=metric_params,
            mode="distance",
            n_jobs=n_jobs,
        )

    #################################
    time_diff_find_neigh = time.time() - start_time_find_neigh
    #################################

    #################################
    start_time_connect_neigh = time.time()
    #################################

    # compute the number of connected components
    # connect the different components
    n_connected_components, labels = connected_components(nbg)
    if n_connected_components > 1:
        if metric == "precomputed" and issparse(X):
            raise RuntimeError(
                "The number of connected components of the neighbors graph"
                f" is {n_connected_components} > 1. The graph cannot be "
                "completed with metric='precomputed'."
                "Please, increase the number of neighbors to avoid this "
                "issue, or precompute the full distance matrix instead "
                "of passing a sparse neighbors graph."
            )
        warnings.warn(
            "The number of connected components of the neighbors graph "
            f"is {n_connected_components} > 1. Completing the graph to fit"
            " Isomap might be slow. Please, increase the number of neighbors "
            "to overcome this issue.",
            stacklevel=2,
        )

        # use array validated by NearestNeighbors
        nbg = _fix_connected_components(
            X=nbrs_._fit_X,
            graph=nbg,
            n_connected_components=n_connected_components,
            component_labels=labels,
            mode="distance",
            metric=nbrs_.effective_metric_,
            **nbrs_.effective_metric_params_,
        )

    #################################
    time_diff_connect_neigh = time.time() - start_time_connect_neigh
    #################################

    
    #################################
    start_time_calc_shortest_path = time.time()
    #################################

    # compute the distance matrix by using the shortest path
    dist_matrix_ = shortest_path(nbg, method=path_method, directed=False)


    #################################
    time_diff_calc_shortest_path = time.time() - start_time_calc_shortest_path
    #################################

    #################################
    time_diff_overall = time.time() - start_time_overall
    #################################
    
    return dist_matrix_, time_diff_overall, time_diff_find_neigh, time_diff_connect_neigh, time_diff_calc_shortest_path

In [4]:
n_iterations = 30
sum_time_diff_overall = 0
sum_time_diff_find_neigh = 0
sum_time_diff_connect_neigh = 0
sum_time_diff_calc_shortest_path = 0

for i in range(n_iterations):
    dist_matr, time_diff_overall, time_diff_find_neigh, time_diff_connect_neigh, time_diff_calc_shortest_path = compute_geodesic_distances_timing(X_california)
    sum_time_diff_overall += time_diff_overall
    sum_time_diff_find_neigh += time_diff_find_neigh
    sum_time_diff_connect_neigh += time_diff_connect_neigh
    sum_time_diff_calc_shortest_path += time_diff_calc_shortest_path
    print('done with iteration ', i)

average_time_diff_overall = sum_time_diff_overall / n_iterations
average_time_diff_find_neigh = sum_time_diff_find_neigh / n_iterations
average_time_diff_connect_neigh = sum_time_diff_connect_neigh / n_iterations
average_time_diff_calc_shortest_path = sum_time_diff_calc_shortest_path / n_iterations

print('overall: ', average_time_diff_overall)
print('finding nearest neighbors: ', average_time_diff_find_neigh)
print('connecting neighbors: ', average_time_diff_connect_neigh)
print('calculating shortest path: ', average_time_diff_calc_shortest_path)

done with iteration  0
done with iteration  1
done with iteration  2
done with iteration  3
done with iteration  4
done with iteration  5
done with iteration  6
done with iteration  7
done with iteration  8
done with iteration  9
done with iteration  10
done with iteration  11
done with iteration  12
done with iteration  13
done with iteration  14
done with iteration  15
done with iteration  16
done with iteration  17
done with iteration  18
done with iteration  19
done with iteration  20
done with iteration  21
done with iteration  22
done with iteration  23
done with iteration  24
done with iteration  25
done with iteration  26
done with iteration  27
done with iteration  28
done with iteration  29
overall:  29.023597097396852
finding nearest neighbors:  0.04430230458577474
connecting neighbors:  0.0012610276540120442
calculating shortest path:  28.978033765157065


## Timing the second version of geodesic distances (with neighbors from pynndescent)

In [5]:
import warnings
import numpy as np
from scipy.sparse import issparse, csr_matrix
from scipy.sparse.csgraph import shortest_path
from scipy.sparse.csgraph import connected_components
from sklearn.neighbors import NearestNeighbors, kneighbors_graph, radius_neighbors_graph
from sklearn.utils.graph import _fix_connected_components


def compute_geodesic_distances_new_neigh_timing(
    X, 
    n_neighbors=10, 
    radius=None,
    path_method="auto", 
    neighbors_algorithm="auto", 
    n_jobs=None, 
    metric="minkowski", 
    p=2, 
    metric_params=None):

    #################################
    start_time_overall = time.time()
    #################################

    #################################
    start_time_find_neigh = time.time()
    #################################

    #################################
    start_time_pynn = time.time()
    #################################

    # first step: find the nearest neighbours
    index = pynndescent.NNDescent(X, n_neighbors=n_neighbors + 1)
    neighbors, distances = index.neighbor_graph

    #################################
    time_diff_pynn = time.time() - start_time_pynn
    #################################

    #################################
    start_time_transform_neigh = time.time()
    #################################

    A_ind = neighbors
    A_data = np.ravel(distances)

    n_samples = X.shape[0]
    n_nonzero = n_samples * n_neighbors
    A_indptr = np.arange(0, n_nonzero + 1, n_neighbors)

    # we want to remove the element itself from the indices of the neighbors
    new_indices_neigh = []
    for i in range(A_ind.shape[0]):
        old_A_ind = A_ind[i]
        old_A_ind = old_A_ind[1:]
        A_ind_new = old_A_ind
        new_indices_neigh.append(A_ind_new)
    new_indices_neigh = np.array(new_indices_neigh)


    # change A_data (remove the distance to itself from the array for each sample)
    nn1 = n_neighbors + 1
    indices_keep = np.arange(nn1 * X.shape[0])
    new_indices_dist = []
    for i in indices_keep:
        if i % nn1 != 0:
            new_indices_dist.append(i)
    new_indices_dist = np.array(new_indices_dist)

    # apply this filtering to the A_data
    new_data = [A_data[i] for i in new_indices_dist]

    #################################
    time_diff_transform_neigh = time.time() - start_time_transform_neigh
    #################################

    nbg = csr_matrix((new_data, new_indices_neigh.ravel(), A_indptr), shape=(n_samples, n_samples))
    

    #################################
    time_diff_find_neigh = time.time() - start_time_find_neigh
    #################################

    #################################
    start_time_connect_neigh = time.time()
    #################################
    
    # compute the number of connected components
    # connect the different components
    n_connected_components, labels = connected_components(nbg)
    if n_connected_components > 1:
        if metric == "precomputed" and issparse(X):
            raise RuntimeError(
                "The number of connected components of the neighbors graph"
                f" is {n_connected_components} > 1. The graph cannot be "
                "completed with metric='precomputed'."
                "Please, increase the number of neighbors to avoid this "
                "issue, or precompute the full distance matrix instead "
                "of passing a sparse neighbors graph."
            )
        warnings.warn(
            "The number of connected components of the neighbors graph "
            f"is {n_connected_components} > 1. Completing the graph to fit"
            " Isomap might be slow. Please, increase the number of neighbors "
            "to overcome this issue.",
            stacklevel=2,
        )

        # use array validated by NearestNeighbors
        nbg = _fix_connected_components(
            X=X,
            graph=nbg,
            n_connected_components=n_connected_components,
            component_labels=labels,
            mode="distance",
            metric=metric
        )

    #################################
    time_diff_connect_neigh = time.time() - start_time_connect_neigh
    #################################

    #################################
    start_time_calc_shortest_path = time.time()
    #################################


    # compute the distance matrix by using the shortest path
    dist_matrix_ = shortest_path(nbg, method=path_method, directed=False)


    #################################
    time_diff_calc_shortest_path = time.time() - start_time_calc_shortest_path
    #################################

    #################################
    time_diff_overall = time.time() - start_time_overall
    #################################
    
    return dist_matrix_, time_diff_overall, time_diff_find_neigh, time_diff_connect_neigh, time_diff_calc_shortest_path, time_diff_transform_neigh, time_diff_pynn

### look at only one specific example

We can see that the part where we use the pynndescent, takes about 15 seconds to run. 

The interesting fact is, that it only takes so long for the first run.

In [6]:
new_dist_matr_single_run, new_time_diff_overall_single_run, new_time_diff_find_neigh_single_run, new_time_diff_connect_neigh_single_run, new_time_diff_calc_shortest_path_single_run, new_time_diff_transform_neigh_single_run, new_time_diff_pynn_single_run = compute_geodesic_distances_new_neigh_timing(X_california)

print('single run:')
print()
print('overall: ', new_time_diff_overall_single_run)
print('find neighbors: ', new_time_diff_find_neigh_single_run)
print('connect neighbors: ', new_time_diff_connect_neigh_single_run)
print('calculate shortest path: ', new_time_diff_calc_shortest_path_single_run)
print('transform neighbors: ', new_time_diff_transform_neigh_single_run)
print('pynn: ', new_time_diff_pynn_single_run)

single run:

overall:  52.8481502532959
find neighbors:  24.482808113098145
connect neighbors:  0.001001596450805664
calculate shortest path:  28.36434054374695
transform neighbors:  0.05200386047363281
pynn:  24.426296710968018


In [7]:
n_iterations = 30
new_sum_time_diff_overall = 0
new_sum_time_diff_find_neigh = 0
new_sum_time_diff_connect_neigh = 0
new_sum_time_diff_calc_shortest_path = 0
new_sum_time_diff_transform_neigh = 0
new_sum_time_diff_pynn = 0

for i in range(n_iterations):
    new_dist_matr, new_time_diff_overall, new_time_diff_find_neigh, new_time_diff_connect_neigh, new_time_diff_calc_shortest_path, new_time_diff_transform_neigh, new_time_diff_pynn = compute_geodesic_distances_new_neigh_timing(X_california)
    new_sum_time_diff_overall += new_time_diff_overall
    new_sum_time_diff_find_neigh += new_time_diff_find_neigh
    new_sum_time_diff_connect_neigh += new_time_diff_connect_neigh
    new_sum_time_diff_calc_shortest_path += new_time_diff_calc_shortest_path
    new_sum_time_diff_transform_neigh += new_time_diff_transform_neigh
    new_sum_time_diff_pynn += new_time_diff_pynn
    print('done with iteration ', i)

new_average_time_diff_overall = new_sum_time_diff_overall / n_iterations
new_average_time_diff_find_neigh = new_sum_time_diff_find_neigh / n_iterations
new_average_time_diff_connect_neigh = new_sum_time_diff_connect_neigh / n_iterations
new_average_time_diff_calc_shortest_path = new_sum_time_diff_calc_shortest_path / n_iterations
new_average_time_diff_transform_neigh = new_sum_time_diff_transform_neigh / n_iterations
new_average_time_diff_pynn = new_sum_time_diff_pynn / n_iterations

print(new_average_time_diff_overall)
print(new_average_time_diff_find_neigh)
print(new_average_time_diff_connect_neigh)
print(new_average_time_diff_calc_shortest_path)
print(new_average_time_diff_transform_neigh)
print(new_average_time_diff_pynn)

done with iteration  0
done with iteration  1
done with iteration  2
done with iteration  3
done with iteration  4
done with iteration  5
done with iteration  6
done with iteration  7
done with iteration  8
done with iteration  9
done with iteration  10
done with iteration  11
done with iteration  12
done with iteration  13
done with iteration  14
done with iteration  15
done with iteration  16
done with iteration  17
done with iteration  18
done with iteration  19
done with iteration  20
done with iteration  21
done with iteration  22
done with iteration  23
done with iteration  24
done with iteration  25
done with iteration  26
done with iteration  27
done with iteration  28
done with iteration  29
29.0457186460495
0.16694064935048422
0.001961342493693034
28.87681665420532
0.053988289833068845
0.10894312858581542
